<a href="https://colab.research.google.com/github/juglar-diaz/SentimentAnalysis/blob/master/SentiPred_SocMedia_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=65a3b0f92e898a22d9d3ef46850f8a8bcb861568918110c0a986b5e70fa4b44a
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [0]:
#!pip install -q tensorflow-text

In [0]:
!pip install -q tensorflow-gpu==2.0.0-beta1
#!pip install -q tf-nightly-gpu-2.0-preview


     |████████████████████████████████| 348.9MB 69kB/s 
     |████████████████████████████████| 501kB 42.3MB/s 
     |████████████████████████████████| 3.1MB 32.2MB/s 


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [0]:
from nltk.tokenize import word_tokenize
from sklearn.model_selection import KFold, train_test_split

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics

import warnings; warnings.simplefilter('ignore')
from collections import Counter
from stop_words import get_stop_words


In [0]:
import stop_words

In [0]:
import collections


import unicodedata
import re
import numpy as np
import os
import io
import itertools


import random
import pickle
import os
sep = os.sep
import os.path

import pandas as pd
import bisect
import time
import scipy.stats as stats

import datetime

import glob
from sklearn import metrics


In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

#Download Data

In [0]:
path_data = 'drive/My Drive/Colab Notebooks/RemoteML/Data/'

In [0]:
all_files = glob.glob(path_data + "/*.json")

li = []

for filename in all_files:
    df = pd.read_json(filename)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [0]:
print(set(df["sentiment"]))

{'', nan, 'negative', 'neutral', 'positive', 'not sure'}


In [0]:
df1 = df[~((df["sentiment"]== ""))]

In [0]:
df1.dropna(subset=['sentiment'],inplace=True)

In [0]:
print(set(df1["sentiment"]))


{'neutral', 'positive', 'negative', 'not sure'}


In [0]:
#df1.info()

In [0]:
df1.reset_index(inplace=True)

In [0]:
X = list(df1["content"])
le = preprocessing.LabelEncoder()
y = le.fit_transform(list(df1["sentiment"]))

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, stratify=y)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.2, stratify=ytrain)

In [0]:
Xtrain[:5]

['De eerste nieuwe uitlopers zijn er vast aan gehangen  hij gaat als een trein. Met beide planten heel blij  allebei rare soorten. Ik ga nog wel aluin begraven bij de hortensia  hopelijk komt de blauwe kleur dan ook hier terug.',
 'Het beste nieuws voor de mensheid sinds tijden by far. Genetische modificatie wordt toegestaan https://fd.nl/economie-politiek/1185108/blokkade-van-genetische-manipulatie-wankelt #GMO',
 'Dat was nu juist datgene dat uit voorzorgsprincipe eerst maar eens grondig onderzocht moet worden voordat de mensheid hieraan wordt blootgesteld. Mijn conclusie gaat dan ook over de biodiversiteit: \nDuidelijk is dat GMO de natuurlijke gewassen aan het verdingen is  Een conclusie die vele wetenschappers zullen kunnen onderschrijven. Boeren worden door ongewenste bestuiving van natuurlijke gewassen door GMO beschuldigd van patent schendingen.',
 'En dat zal altijd zo zijn. | Title: Grootschalig onderzoek: GMO-voedsel veilig om te eten',
 'RT @Maaike_Bionext A. O. Biotechnolo

In [0]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    Xtrain, target_vocab_size=5000)


In [0]:
sample_string = 'Veel beter is het dikke lookbollen uit Arleux te kopen'

tokenized_string = tokenizer.encode(sample_string)
print ('Tokenized string is {}'.format(tokenized_string))

original_string = tokenizer.decode(tokenized_string)
print ('The original string: {}'.format(original_string))

assert original_string == sample_string

Tokenized string is [3195, 416, 10, 7, 4607, 961, 3007, 354, 177, 42, 438, 1631, 1558, 11, 746]
The original string: Veel beter is het dikke lookbollen uit Arleux te kopen


In [0]:
Xtraintoke = [tokenizer.encode(s) for s in Xtrain]
train = [(text,ytrain[i]) for (i,text) in enumerate(Xtraintoke)]
train_dataset = tf.data.Dataset.from_generator(lambda: train, (tf.int32, tf.int32), (tf.TensorShape([None]), tf.TensorShape([])))


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [0]:
Xvaltoke = [tokenizer.encode(s) for s in Xval]
val = [(text,yval[i]) for (i,text) in enumerate(Xvaltoke)]
val_dataset = tf.data.Dataset.from_generator(lambda: val, (tf.int32, tf.int32), (tf.TensorShape([None]), tf.TensorShape([])))


In [0]:
Xtesttoke = [tokenizer.encode(s) for s in Xtest]
test = [(text,ytest[i]) for (i,text) in enumerate(Xtesttoke)]
test_dataset = tf.data.Dataset.from_generator(lambda: test, (tf.int32, tf.int32), (tf.TensorShape([None]), tf.TensorShape([])))


In [0]:
BUFFER_SIZE = 5000
BATCH_SIZE = 32
MAX_LENGTH = 80

In [0]:
k = MAX_LENGTH
def pad_or_trunc(t,s):
    
    dim = tf.size(t)
    return (tf.cond(tf.equal(dim, k), lambda: t, lambda: tf.cond(tf.greater(dim, k), lambda: tf.slice(t, [0], [k]), lambda: tf.concat([t, tf.zeros(k-dim, dtype=tf.int32)], 0))),s)


In [0]:
#train_dataset = train_dataset.map(pad_or_trunc)

In [0]:
# cache the dataset to memory to get a speedup while reading from it.
#train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None],[]))
train_dataset = train_dataset.repeat(40)



In [0]:
batch = next(iter(train_dataset))
batch

(<tf.Tensor: id=99, shape=(32, 93), dtype=int32, numpy=
 array([[ 361,  703,   95, ...,    0,    0,    0],
        [  59,   31, 3756, ..., 3268, 5040, 4993],
        [1695, 2983, 4972, ...,    0,    0,    0],
        ...,
        [1706, 3410, 3987, ...,    0,    0,    0],
        [5022, 5025, 5025, ...,    0,    0,    0],
        [4806, 1394, 3815, ...,    0,    0,    0]], dtype=int32)>,
 <tf.Tensor: id=100, shape=(32,), dtype=int32, numpy=
 array([1, 3, 1, 1, 1, 1, 1, 1, 0, 3, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 3,
        1, 3, 3, 1, 1, 1, 1, 1, 3, 1], dtype=int32)>)

In [0]:
val_dataset = val_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None],[]))
test_dataset = test_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

## Create the model

## Positional encoding

Since this model doesn't contain any recurrence or convolution, positional encoding is added to give the model some information about the relative position of the words in the sentence. 

The positional encoding vector is added to the embedding vector. Embeddings represent a token in a d-dimensional space where tokens with similar meaning will be closer to each other. But the embeddings do not encode the relative position of words in a sentence. So after adding the positional encoding, words will be closer to each other based on the *similarity of their meaning and their position in the sentence*, in the d-dimensional space.

See the notebook on [positional encoding](https://github.com/tensorflow/examples/blob/master/community/en/position_encoding.ipynb) to learn more about it. The formula for calculating the positional encoding is as follows:

$$\Large{PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})} $$
$$\Large{PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})} $$

In [0]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [0]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
  pos_encoding = angle_rads[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)

## Masking

Mask all the pad tokens in the batch of sequence. It ensures that the model does not treat padding as the input. The mask indicates where pad value `0` is present: it outputs a `1` at those locations, and a `0` otherwise.

In [0]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

## Scaled dot product attention

In [0]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

As the softmax normalization is done on K, its values decide the amount of importance given to Q.

The output represents the multiplication of the attention weights and the V (value) vector. This ensures that the words you want to focus on are kept as-is and the irrelevant words are flushed out.

In [0]:
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print ('Attention weights are:')
  print (temp_attn)
  print ('Output is:')
  print (temp_out)

## Multi-head attention

Each multi-head attention block gets three inputs; Q (query), K (key), V (value). These are put through linear (Dense) layers and split up into multiple heads. 

The `scaled_dot_product_attention` defined above is applied to each head (broadcasted for efficiency). An appropriate mask must be used in the attention step.  The attention output for each head is then concatenated (using `tf.transpose`, and `tf.reshape`) and put through a final `Dense` layer.

Instead of one single attention head, Q, K, and V are split into multiple heads because it allows the model to jointly attend to information at different positions from different representational spaces. After the split each head has a reduced dimensionality, so the total computation cost is the same as a single head attention with full dimensionality.

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

Create a `MultiHeadAttention` layer to try out. At each location in the sequence, `y`, the `MultiHeadAttention` runs all 8 attention heads across all other locations in the sequence, returning a new vector of the same length at each location.

In [0]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

(TensorShape([1, 60, 512]), TensorShape([1, 8, 60, 60]))

## Point wise feed forward network

Point wise feed forward network consists of two fully-connected layers with a ReLU activation in between.

In [0]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [0]:
sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512))).shape

TensorShape([64, 50, 512])

### Encoder layer

Each encoder layer consists of sublayers:

1.   Multi-head attention (with padding mask) 
2.    Point wise feed forward networks. 

Each of these sublayers has a residual connection around it followed by a layer normalization. Residual connections help in avoiding the vanishing gradient problem in deep networks.

The output of each sublayer is `LayerNorm(x + Sublayer(x))`. The normalization is done on the `d_model` (last) axis. There are N encoder layers in the transformer.

In [0]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [0]:
sample_encoder_layer = EncoderLayer(512, 8, 2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((64, 43, 512)), False, None)

sample_encoder_layer_output.shape  # (batch_size, input_seq_len, d_model)

TensorShape([64, 43, 512])

### Encoder

The `Encoder` consists of:
1.   Input Embedding
2.   Positional Encoding
3.   N encoder layers

The input is put through an embedding which is summed with the positional encoding. The output of this summation is the input to the encoder layers. The output of the encoder is the input to the decoder.

In [0]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(input_vocab_size, self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)

In [0]:
sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, input_vocab_size=8500)

sample_encoder_output = sample_encoder(tf.random.uniform((64, 62)), 
                                       training=False, mask=None)

print (sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

(64, 62, 512)


## Create the Transformer

Transformer consists of the encoder, decoder and a final linear layer. The output of the decoder is the input to the linear layer and its output is returned.

In [0]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               classes_size, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                           input_vocab_size, rate)

    #self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
    #                       target_vocab_size, rate)
    
    self.rnn = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(d_model))
    
   

    self.final_layer = tf.keras.layers.Dense(classes_size, activation='softmax')
    
  def call(self, inp, training, enc_padding_mask):

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
    #print(enc_output.shape)
    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    output = self.rnn(enc_output)
    #print(output.shape)
    
    final_output = self.final_layer(output)  # (batch_size, tar_seq_len, target_vocab_size)
    
    return final_output

In [0]:
sample_transformer = Transformer(
    num_layers=1, d_model=512, num_heads=8, dff=2048, 
    input_vocab_size=8500, classes_size=4)


temp_input = tf.random.uniform((64, 62))
fn_out = sample_transformer(temp_input, training=False, 
                               enc_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

TensorShape([64, 4])

## Set hyperparameters

To keep this example small and relatively fast, the values for *num_layers, d_model, and dff* have been reduced. 

The values used in the base model of transformer were; *num_layers=6*, *d_model = 512*, *dff = 2048*. See the [paper](https://arxiv.org/abs/1706.03762) for all the other versions of the transformer.

Note: By changing the values below, you can get the model that achieved state of the art on many tasks.

In [0]:
num_layers = 1
d_model = 128
dff = 256
num_heads = 8

input_vocab_size = tokenizer.vocab_size + 2
classes_size=4
dropout_rate = 0.1

## Optimizer

Use the Adam optimizer with a custom learning rate scheduler according to the formula in the [paper](https://arxiv.org/abs/1706.03762).

$$\Large{lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$


In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

## Loss and metrics

Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [0]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

## Training and checkpointing

In [0]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, classes_size, dropout_rate)

In [0]:
def create_masks(inp):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)
  return enc_padding_mask

Create the checkpoint path and the checkpoint manager. This will be used to save checkpoints every `n` epochs.

In [0]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
#if ckpt_manager.latest_checkpoint:
#  ckpt.restore(ckpt_manager.latest_checkpoint)
#  print ('Latest checkpoint restored!!')

The target is divided into tar_inp and tar_real. tar_inp is passed as an input to the decoder. `tar_real` is that same input shifted by 1: At each location in `tar_input`, `tar_real` contains the  next token that should be predicted.

For example, `sentence` = "SOS A lion in the jungle is sleeping EOS"

`tar_inp` =  "SOS A lion in the jungle is sleeping"

`tar_real` = "A lion in the jungle is sleeping EOS"

The transformer is an auto-regressive model: it makes predictions one part at a time, and uses its output so far to decide what to do next. 

During training this example uses teacher-forcing (like in the [text generation tutorial](./text_generation.ipynb)). Teacher forcing is passing the true output to the next time step regardless of what the model predicts at the current time step.

As the transformer predicts each word, *self-attention* allows it to look at the previous words in the input sequence to better predict the next word.

To prevent the model from peaking at the expected output the model uses a look-ahead mask.

In [0]:
EPOCHS = 1

In [0]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  loss = 0
  enc_padding_mask= create_masks(inp)
  
  with tf.GradientTape() as tape:
    predictions = transformer(inp, 
                                 True, 
                                 enc_padding_mask)
    loss = loss_function(tar, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  print(loss)
  train_loss(loss)
  train_accuracy(tar, predictions)

In [0]:
for epoch in range(EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  # inp -> portuguese, tar -> english
  for (batch, (inp, tar)) in enumerate(train_dataset):

    train_step(inp, tar)
    
    if batch % 10 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
      
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Tensor("Mean:0", shape=(), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
Epoch 1 Batch 0 Loss 1.2592 Accuracy 0.0938
Epoch 1 Batch 10 Loss 1.3235 Accuracy 0.1705
Epoch 1 Batch 20 Loss 1.3246 Accuracy 0.2024
Epoch 1 Batch 30 Loss 1.3081 Accuracy 0.2722
Epoch 1 Batch 40 Loss 1.2945 Accuracy 0.3590
Epoch 1 Batch 50 Loss 1.2779 Accuracy 0.4252
Epoch 1 Batch 60 Loss 1.2492 Accuracy 0.4688
Epoch 1 Batch 70 Loss 1.2257 Accuracy 0.4915
Epoch 1 Batch 80 Loss 1.1971 Accuracy 0.5199
Epoch 1 Batch 90 Loss 1.1767 Accuracy 0.5339
Epoch 1 Batch 100 Loss 1.1637 Accuracy 0.5449
Epoch 1 Batch 110 Loss 1.1470 Accuracy 0.5556
Epoch 1 Batch 120 Loss 1.1333 Accuracy 0.5668
Epoch 1 Batch 130 Loss 1.1197 Accuracy 0.5761
Epoch 1 Batch 140 Loss 1.1122 Accuracy 0.5813
Epoch 1 Batch 150 Loss 1.1026 Accuracy 0.5888
Epoch 1 Batch 160 Loss 1.0945 Accuracy 0.5946
Epoch 1 Batch 170 Loss 1.0897 Accuracy 0.5973
Epoch 1 Batch 180 Loss 1.0842 Accuracy 0.6034
Epoch 1 Batch 190 Loss 1.0756 Accuracy 0.6069
Epoch 1